# Math expressions - audio dataset

In [2]:
from datasets import Dataset
import torchaudio
import os
import pandas as pd
from tqdm import tqdm

c:\Users\User\Documents\Projets\FST-For-LaTeX\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Dataset loader

In [3]:
class MathDataset(Dataset):
    
    def __init__(self, transform=None):
        self.audio_dir = "Data\\audio"
        self.langage_dir = "Data\\sequences"
        self.latex_dir = "Data\\latex"
        self.transform = transform

        # Liste des fichiers communs
        self.audio_files = [f for f in os.listdir(self.audio_dir) if f.endswith('.wav')]
        self.langage_files = [f for f in os.listdir(self.langage_dir) if f.endswith('.txt')]
        self.latex_files = [f for f in os.listdir(self.latex_dir) if f.endswith('.txt')]

        # Assurez-vous que les fichiers sont correspondants
        self.common_files = list(set([f.split('.')[0] for f in self.audio_files]).intersection(
                                 set([f.split('.')[0] for f in self.langage_files]),
                                 set([f.split('.')[0] for f in self.latex_files])))

    def __len__(self):
        return len(self.common_files)

    def __getitem__(self, idx):
    
        file_id = self.common_files[idx]

        audio_path = os.path.join(self.audio_dir, f"{file_id}.wav")
        langage_path = os.path.join(self.langage_dir, f"{file_id}.txt")
        latex_path = os.path.join(self.latex_dir, f"{file_id}.txt")

        # audio_waveform, sample_rate = torchaudio.load(audio_path)

        with open(langage_path, 'r') as f:
            langage_text = f.read().split('\n')[0]

        with open(latex_path, 'r') as f:
            latex_text = f.read().split('\n')[0]

        if self.transform:
            audio_waveform = self.transform(audio_waveform)

        sample = {
            'id': file_id,
            'path': audio_path,
            'audio': audio_waveform,
            # 'sample_rate': sample_rate,
            'langage': langage_text,
            'latex': latex_text
        }

        return sample

    def get_pandas_df(self):

        ids = []
        path = []
        sample_rates = []
        seq = []
        tex = []
        dna = []
        user_ids = []

        
        for idx in tqdm(range(len(self.common_files))): 
            
            file_id = self.common_files[idx] 
            

            for i in range(100):
                if 'p'+str(i) in file_id : user_ids.append(i)

            


            
            audio_path = os.path.join(self.audio_dir, f"{file_id}.wav")
            langage_path = os.path.join(self.langage_dir, f"{file_id}.txt")
            latex_path = os.path.join(self.latex_dir, f"{file_id}.txt")
    
            # audio_waveform, sample_rate = torchaudio.load(audio_path)
    
            with open(langage_path, 'r') as f:
                langage_text = f.read().split('\n')[0]
    
            with open(latex_path, 'r') as f:
                latex_text = f.read().split('\n')[0]

            ids.append(file_id)
            dna.append('dna' in file_id)
            path.append(audio_path)
            # sample_rates.append(sample_rate)
            seq.append(langage_text)
            tex.append(latex_text)

        return pd.DataFrame({
            'id':ids,
            'user_id':user_ids,
            'path':path,
            # 'sample_rate':sample_rates,
            'seq':seq,
            'tex':tex,
            'dna':dna,
        })
            



dataset = MathDataset()

In [4]:
df = dataset.get_pandas_df()

100%|██████████| 245/245 [00:00<00:00, 2022.96it/s]


In [5]:
df.loc[df['dna'] == True]

,id,user_id,path,seq,tex,dna
1,line1047_p1_dna,1,Data\audio\line1047_p1_dna.wav,somme pour i égal un à grand n moins s moins d...,\sum _ { i = 1 } ^ { N - s - 2 } \alpha _ i \l...,True
2,line770_p1_dna,1,Data\audio\line770_p1_dna.wav,degré indice alpha parenthèse grand f indice g...,\deg _ { \alpha } ( F _ H \circ g ) \geq ( d -...,True
6,line876_p2_dna,2,Data\audio\line876_p2_dna.wav,grand f indice grand h égal grand x indice zér...,"F _ H = X _ 0 P _ 0 + \dots + X _ N P _ N ,",True
7,line935_p1_dna,1,Data\audio\line935_p1_dna.wav,d ronde indice accolade grand x indice accolad...,\partial _ { X _ { i _ 0 } } F _ H = P _ { i _...,True
16,line761_p1_dna,1,Data\audio\line761_p1_dna.wav,degré indice alpha parenthèse grand f indice g...,\deg _ { \alpha } ( F _ H \circ g ) \geq ( d -...,True
...,...,...,...,...,...,...
234,line896_p2_dna,2,Data\audio\line896_p2_dna.wav,alpha indice i plus somme indice accolade j ég...,\alpha _ i + \sum _ { j = 0 } ^ N m _ j \alpha...,True
238,line821_p1_dna,1,Data\audio\line821_p1_dna.wav,supérieur ou égal à parenthèse d moins deux de...,\geq ( d - 2 \delta _ P + 1 ) \alpha _ N - ( \...,True
239,line1058_p2_dna,2,Data\audio\line1058_p2_dna.wav,supérieur ou égal à parenthèse d moins deux de...,\geq ( d - 2 \delta _ P ) \alpha _ N - \delta ...,True
241,line786_p1_dna,1,Data\audio\line786_p1_dna.wav,degré indice alpha parenthèse grand f indice g...,"\deg _ { \alpha } ( F _ H \circ g ) \geq d \, ...",True


In [6]:
from src.FST.transducers import *
tex_normalizer = TexNormalizer("src/FST/rules/tex_normalizer.txt")

In [7]:
tex_norm = [tex_normalizer.predict(i).replace('$','') for i in df['tex']]
df['tex'] = tex_norm

In [8]:
df

,id,user_id,path,seq,tex,dna
0,line1119_p1,1,Data\audio\line1119_p1.wav,d supérieur ou égal à grand n plus un facteur ...,d \geq ( N + 1 ) ( \delta - 1 ) \quad ( \text ...,False
1,line1047_p1_dna,1,Data\audio\line1047_p1_dna.wav,somme pour i égal un à grand n moins s moins d...,\sum _ { i = 1 } ^ { N - s - 2 } \alpha _ i \l...,True
2,line770_p1_dna,1,Data\audio\line770_p1_dna.wav,degré indice alpha parenthèse grand f indice g...,\deg _ { \alpha } ( F _ H \circ g ) \geq ( d -...,True
3,line172_p3,3,Data\audio\line172_p3.wav,probabilité entre parenthèses p indice t entre...,\mathbb { P } ( p _ \mathcal { T } ( [ T _ { b...,False
4,line149_p3,3,Data\audio\line149_p3.wav,égal esperance de l'indicatrice de y h plus gr...,= \mathbb { E } \bigg [ \mathbf { 1 } _ { \{ Y...,False
...,...,...,...,...,...,...
240,line352_p3,3,Data\audio\line352_p3.wav,grand d tilde indice un entre parenthèses gran...,\tilde { D } _ 1 ( B _ \varepsilon ( \tilde {...,False
241,line786_p1_dna,1,Data\audio\line786_p1_dna.wav,degré indice alpha parenthèse grand f indice g...,"\deg _ { \alpha } ( F _ H \circ g ) \geq d \, ...",True
242,line966_p1_dna,1,Data\audio\line966_p1_dna.wav,inférieur ou égal à dim parenthèse grand h ind...,\leq \dim \big ( H _ { \mathrm { sing } } \cap...,True
243,line976_p1,1,Data\audio\line976_p1.wav,s deux points égal dim de h sing dans l'ensemb...,s : = \dim ( H _ { \mathrm { sing } } ) \in \{...,False


In [9]:
mathFST = MathTransducer("src/FST/rules/math_rules.txt", "src/FST/rules/normalizer.txt", "src/FST/rules/lemmatizer.txt")
tex_pred = [mathFST.predict(i) for i in df['seq']]

In [10]:
tex_pred

['d \\geq N + 1 \\delta - 1 d > N + 1 \\delta - 1',
 '\\sum i = 1 N - s - 2 \\alpha _ i \\leq 0',
 '\\deg _ \\alpha ( F _ H \\circ g ) \\geq ( d - 2 \\delta _ P + 1 ) \\alpha _ N - ( \\delta _ P - 2 ) \\alpha _ 1 - ( \\delta _ P - 1 ) \\sum i = 2 N - 1 \\alpha _ i',
 '\\mathbb { P } ( p _ t ( [ T _ b - \\epsilon , T _ b \\cup [ t _ b , t _ b - \\epsilon \\subset A _ L \\epsilon \\cup a _ l \\epsilon > 1 - \\delta',
 '= \\mathbb { E } y h > \\alpha \\epsilon \\exp - 2 \\int S _ \\alpha \\epsilon S _ b N _ x t 0 < \\omega ^ * w ^ * < \\epsilon',
 '\\deg _ \\alpha F : = \\max m \\in N \\mathbb { N } N + 1 m = d \\lambda _ m \\neq 0 \\deg _ \\alpha x M',
 'F _ H = X _ 0 P _ 0 + \\dots + X _ N P _ N',
 '\\partial _ { X _ { i _ 0 } } F _ H = P _ { i _ 0 } + \\sum i = 0 N X _ i \\partial _ { X _ { i _ 0 } } P _ i',
 '\\geq n 0 y h > \\alpha \\epsilon \\vert w ^ * < - b ( \\mathbb { E } i \\alpha < x \\exp - 3 i \\alpha + \\mathbb { E } i \\alpha \\geq x \\exp - 3 i \\alpha )',
 'd ( \\gamma l

In [11]:
from src.Metrics.WordErrorRate import Metrics

In [12]:
WER_FST = []
for i, t in enumerate(df['tex']) : 
    ref = t
    hyp = tex_pred[i]
    WER_FST.append(Metrics.WER(ref, hyp))

In [13]:
df['WER_FST'] = WER_FST

In [14]:
df.loc[df['dna'] == True, 'WER_FST'].mean()

0.10622257800769577

In [15]:
df.loc[df['dna'] == False, 'WER_FST'].mean()

0.4766254858422202

In [16]:
tex_lex = [i.replace('}', '').replace('{', '').replace('(', '').replace(')', '').replace('[', '').replace(']', '').replace('\\{', '').replace('\\}', '') for i in df['tex']]
    

In [17]:
tex_lex

['d \\geq  N + 1   \\delta - 1  \\quad  \\text  resp.  d >  N + 1   \\delta - 1  ',
 '\\sum _  i = 1  ^  N - s - 2  \\alpha _ i \\leq 0',
 '\\deg _  \\alpha   F _ H \\circ g  \\geq  d - 2 \\delta _ P + 1  \\alpha _ N -  \\delta _ P - 2  \\alpha _ 1 -  \\delta _ P - 1  \\sum _  i = 2  ^  N - 1  \\alpha _ i',
 '\\mathbb  P   p _ \\mathcal  T    T _  b - \\varepsilon  , T _ b  \\cup  \\hat  T  _ b , \\hat  T  _  b - \\varepsilon    \\subset A _  L \\varepsilon  \\cup \\hat  A  _  L \\varepsilon   > 1 - \\delta',
 '= \\mathbb  E  \\bigg  \\mathbf  1  _  \\ Y _ H > \\alpha \\varepsilon \\  \\exp \\bigg  - 2 \\int _  S _  \\alpha \\varepsilon   ^  S _ b  \\mathbb  N  _  X _ t   0 < W _ * < \\varepsilon  d t \\bigg  \\bigg ',
 '\\deg _  \\alpha  F : = \\underset  \\substack  m \\in \\mathbb  N  ^  N + 1  \\\\ \\vert m \\vert = d \\\\ \\lambda _ m \\neq 0    \\mathrm  max   \\deg _  \\alpha  X ^ m',
 'F _ H = X _ 0 P _ 0 + \\dots + X _ N P _ N ,',
 '\\partial _  X _  i _ 0   F _ H = P _  i _ 0

In [18]:
WER_LEX_FST = []
for i in range(len(df['tex'])) : 
    ref = tex_lex[i]
    hyp = tex_pred[i].replace('}', '').replace('{', '').replace('(', '').replace(')', '').replace('[', '').replace(']', '').replace('\\{', '').replace('\\}', '')
    WER_LEX_FST.append(Metrics.WER(ref, hyp))

In [19]:
import numpy as np
np.mean(WER_LEX_FST)

0.252432157793276

In [20]:
df['WER_LEX_FST'] = WER_LEX_FST

In [21]:
df

,id,user_id,path,seq,tex,dna,WER_FST,WER_LEX_FST
0,line1119_p1,1,Data\audio\line1119_p1.wav,d supérieur ou égal à grand n plus un facteur ...,d \geq ( N + 1 ) ( \delta - 1 ) \quad ( \text ...,False,0.483871,0.157895
1,line1047_p1_dna,1,Data\audio\line1047_p1_dna.wav,somme pour i égal un à grand n moins s moins d...,\sum _ { i = 1 } ^ { N - s - 2 } \alpha _ i \l...,True,0.300000,0.125000
2,line770_p1_dna,1,Data\audio\line770_p1_dna.wav,degré indice alpha parenthèse grand f indice g...,\deg _ { \alpha } ( F _ H \circ g ) \geq ( d -...,True,0.131148,0.042553
3,line172_p3,3,Data\audio\line172_p3.wav,probabilité entre parenthèses p indice t entre...,\mathbb { P } ( p _ \mathcal { T } ( [ T _ { b...,False,0.485294,0.309524
4,line149_p3,3,Data\audio\line149_p3.wav,égal esperance de l'indicatrice de y h plus gr...,= \mathbb { E } \bigg [ \mathbf { 1 } _ { \{ Y...,False,0.661765,0.543478
...,...,...,...,...,...,...,...,...
240,line352_p3,3,Data\audio\line352_p3.wav,grand d tilde indice un entre parenthèses gran...,\tilde { D } _ 1 ( B _ \varepsilon ( \tilde {...,False,0.456790,0.368421
241,line786_p1_dna,1,Data\audio\line786_p1_dna.wav,degré indice alpha parenthèse grand f indice g...,"\deg _ { \alpha } ( F _ H \circ g ) \geq d \, ...",True,0.150000,0.062500
242,line966_p1_dna,1,Data\audio\line966_p1_dna.wav,inférieur ou égal à dim parenthèse grand h ind...,\leq \dim \big ( H _ { \mathrm { sing } } \cap...,True,0.187500,0.272727
243,line976_p1,1,Data\audio\line976_p1.wav,s deux points égal dim de h sing dans l'ensemb...,s : = \dim ( H _ { \mathrm { sing } } ) \in \{...,False,0.440000,0.263158


In [22]:
df.loc[df['dna'] == True, 'WER_LEX_FST'].mean()

0.06812502151311932

In [23]:
df.loc[df['dna'] == False, 'WER_LEX_FST'].mean()

0.34346190193164605

In [24]:
voila = pd.read_csv("Data/voila.csv")

In [30]:
 voila[['id', 'whips_med_pred', 'time_med_pred', 'whisp_base_pred', 'whisp_base_time']]

,id,whips_med_pred,time_med_pred,whisp_base_pred,whisp_base_time
0,line966_p1_dna,inferior ou égal à DIM parenthèse grand H à c...,15.755742,1 faillère ou égal à 10 parenthèses Grantage ...,3.727660
1,line1059_p2_dna,supérieur ou égal à parenthèse D moins 2 fois...,34.245990,Superior ou égal à parenthèse des moins deux ...,3.976433
2,line821_p1_dna,supérieur ou égal à parenthèse D moins 2 delt...,35.246260,Superior ou égal à la parenthèse des moins de...,3.900681
3,line1047_p2_dna,Somme indice à collade i = 1 fermé la collade...,18.001053,Somme indice à collade e = 1 fermez la collad...,2.263451
4,line857_p1_dna,dimension parenthèse grand H indice à collade...,42.653508,Dimension parenthèse gantache indice à collad...,5.310216
...,...,...,...,...,...
240,line846_p1,alpha 0 infernale à 30 points infernale à alp...,9.685408,Alpha Zéro inférieure à la 3d point inférieur...,1.366335
241,line832_p1,super ou égal à D - 2 delta P + 1 facteur de ...,18.407689,Super rigarde a des morts de détaper plus en ...,3.237125
242,line190_p3,Grand D tilde puissance rond indice entre par...,27.893562,Grand détail de plus en ce rond indice entre ...,4.199613
243,line158_p3,supérieur ou égal à n0 de yh supérieur à alph...,18.217793,Superior Regal a N0 de Y' supérieure Alpheip ...,2.835896


In [34]:
df2 = pd.merge(df, voila[['id', 'whips_med_pred', 'time_med_pred', 'whisp_base_pred', 'whisp_base_time']], on='id', how='inner')
df2 = df2.rename({
    'whips_med_pred':'y_seq_whisper_medium', 
    'time_med_pred':'time_whisper_medium',
    'whisp_base_pred':'y_seq_whisper_base',
    'whisp_base_time':'time_whisper_base'
}, axis=1)

df2

,id,user_id,path,seq,tex,dna,WER_FST,WER_LEX_FST,y_seq_whisper_medium,time_whisper_medium,y_seq_whisper_base,time_whisper_base
0,line1119_p1,1,Data\audio\line1119_p1.wav,d supérieur ou égal à grand n plus un facteur ...,d \geq ( N + 1 ) ( \delta - 1 ) \quad ( \text ...,False,0.483871,0.157895,des supers ou égales à grante plus 1 facteur ...,13.013499,Des supermégal à la grottaine plus en facteur...,1.982889
1,line1047_p1_dna,1,Data\audio\line1047_p1_dna.wav,somme pour i égal un à grand n moins s moins d...,\sum _ { i = 1 } ^ { N - s - 2 } \alpha _ i \l...,True,0.300000,0.125000,"Somme pour i = 1a / (s - 2), 2a / 10i = 0",9.430227,Somme pour y également un à containe -sme 2 2...,1.604947
2,line770_p1_dna,1,Data\audio\line770_p1_dna.wav,degré indice alpha parenthèse grand f indice g...,\deg _ { \alpha } ( F _ H \circ g ) \geq ( d -...,True,0.131148,0.042553,degré indice alpha parenthèse grand F indice ...,41.459625,Degré un dis alpha parenthèse Grand F indisco...,6.292350
3,line172_p3,3,Data\audio\line172_p3.wav,probabilité entre parenthèses p indice t entre...,\mathbb { P } ( p _ \mathcal { T } ( [ T _ { b...,False,0.485294,0.309524,probabilité entre parenthèses P indice T entr...,24.936835,probabilité entre parenthes P110t entre paren...,3.889057
4,line149_p3,3,Data\audio\line149_p3.wav,égal esperance de l'indicatrice de y h plus gr...,= \mathbb { E } \bigg [ \mathbf { 1 } _ { \{ Y...,False,0.661765,0.543478,égale espérance de l'indicateur de y h plus g...,22.023126,et à l'espérance de l'indiquatrice de l'écréc...,3.353830
...,...,...,...,...,...,...,...,...,...,...,...,...
240,line352_p3,3,Data\audio\line352_p3.wav,grand d tilde indice un entre parenthèses gran...,\tilde { D } _ 1 ( B _ \varepsilon ( \tilde {...,False,0.456790,0.368421,"GNDILDE1 (grand B indice epsilon), STILDE1 (g...",14.380724,Grand détail d'indices 1 entre parenthèses Gr...,11.514585
241,line786_p1_dna,1,Data\audio\line786_p1_dna.wav,degré indice alpha parenthèse grand f indice g...,"\deg _ { \alpha } ( F _ H \circ g ) \geq d \, ...",True,0.150000,0.062500,Degré indice alpha parenthèse grand F indice ...,13.296028,Degrir l'Alpha parenthèse Grotef un discrotec...,1.970876
242,line966_p1_dna,1,Data\audio\line966_p1_dna.wav,inférieur ou égal à dim parenthèse grand h ind...,\leq \dim \big ( H _ { \mathrm { sing } } \cap...,True,0.187500,0.272727,inferior ou égal à DIM parenthèse grand H à c...,15.755742,1 faillère ou égal à 10 parenthèses Grantage ...,3.727660
243,line976_p1,1,Data\audio\line976_p1.wav,s deux points égal dim de h sing dans l'ensemb...,s : = \dim ( H _ { \mathrm { sing } } ) \in \{...,False,0.440000,0.263158,S de point égal 10 de H5 dans l'ensemble -1 v...,11.211691,S de point égal 10 de H5 dans l'ensemble vois...,1.284631


## Normalisation des sorties Whisper

In [38]:
from src.FST.transducers import RulesTransducer

normalizer = RulesTransducer(output_word=False, complete_mod=False, trash_mod=False)


In [39]:
norm_whisp_base = [normalizer.predict(i) for i in df2['y_seq_whisper_base']]

FstStringCompilationError: String compilation failed